In [ ]:
!pip install scikit-optimize
!git clone https://github.com/modestyachts/cifar-10.2.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 2.9 MB/s eta 0:00:00
Cloning into 'cifar-10.2'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import os
import numpy as np
from time import perf_counter
from google.colab import drive
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.models import Model, Sequential, load_model
from keras.backend import clear_session
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Resizing, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy, KLDivergence 
from keras.utils import to_categorical
from tensorflow.keras.applications import inception_v3, vgg16, ResNet50, resnet50  
import skopt
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical, LogN
from skopt.plots import plot_convergence
from skopt import callbacks
from skopt.callbacks import CheckpointSaver


drive.mount('/content/drive')
gdrive_dir = '/content/drive/MyDrive/Colab Notebooks/CI/Logfiles/'  # --change to your path--
assert os.path.exists(gdrive_dir)

tf.random.set_seed(420)
tf.__version__

Mounted at /content/drive


'2.9.2'

In [ ]:
# Cifar 10.2 
def preprocessCIFAR10_2():
  train_data = np.load('cifar-10.2/cifar102_train.npz')
  test_data = np.load('cifar-10.2/cifar102_test.npz')
  label_names = test_data['label_names']

  x_train = train_data['images']
  y_train = train_data['labels']

  indexes_test = np.array(range(len(x_train)))
  random.shuffle(indexes_test)
  x_train = x_train[indexes_test]
  y_train1 = y_train[indexes_test]

  x_test = test_data['images']
  y_test = test_data['labels']

  # One-Hot encoding
  y_train = to_categorical(y_train1, num_classes=10)
  y_test = to_categorical(y_test, num_classes=10)
  return(x_train, y_train, x_test, y_test, label_names)

In [ ]:
def print_learning_curves(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()),1])
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.ylabel('Cross Entropy')
  plt.title('Training and Validation Loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def print_report(model, x, y, labels):

  # Prediction
  predictions = model.predict(x)
  # Select for each observation the highest probability
  predictions = np.argmax(predictions, axis=1)
  real = np.argmax(y, axis=1)
  # Confusion matrix
  confusionMatrix = confusion_matrix(real, predictions)
  print(pd.DataFrame(confusionMatrix, columns=labels, index=labels))

  # Classification Report
  print(classification_report(real, predictions, target_names=labels,))
  plot_confusion_matrix(confusionMatrix, "Final Model Confusion Matrix")


In [ ]:
def create_model(name, nlayers, units, dropout_rate):

  if name == 'resNet50':
    base_model = ResNet50
  elif name == 'inceptionv3':
    base_model = inception_v3.InceptionV3
  elif name == 'vgg16':
    base_model = vgg16.VGG16

  # Defining the model
  base_model = base_model(
      include_top=False,
      weights='imagenet',
      input_shape = (299, 299, 3),
      classes=10
  )

  # Resize + Base Model
  final_model = Sequential()
  final_model.add(Resizing(299, 299))
  final_model.add(base_model)

  # Define new layers
  new_layers = Sequential()
  new_layers.add(GlobalAveragePooling2D(name='avg_pool'))
  for i in range(nlayers):
    u = units * 2 ** -i
    new_layers.add(Dense(u, activation='relu'))
    new_layers.add(Dropout(dropout_rate))
  new_layers.add(Dense(10, activation='softmax'))

  # Joining both models
  final_model.add(new_layers)
  
  return final_model, base_model, new_layers

In [ ]:
def train_model(params, model_name, nlayers, reuse_params=None, eval=False, learning_curve=False):
  clear_session()
  t_start = perf_counter()

  epochs1 = 100
  epochs2 = 100
  patience = 5

  if reuse_params:
    units, dropout = params
    lr, batch_size, loss = reuse_params
  else:
    units, dropout, lr, batch_size, loss = params
    print(units, dropout, lr, batch_size, loss)
  batch_size = 2**batch_size  # use as exponent

  print(f'\nParameter:\n units:{units} dropout:{dropout} lr:{lr} batch_size:{batch_size} loss:{loss}\n')

  # Define all the parameters
  x_train, y_train, x_test, y_test, label_names = preprocessCIFAR10_2()

  # preprocess data
  if model_name == 'resNet50':
    x_tr = resnet50.preprocess_input(x_train)
    x_te = resnet50.preprocess_input(x_test)
  elif model_name == 'inceptionv3':
    x_tr = inception_v3.preprocess_input(x_train)
    x_te = inception_v3.preprocess_input(x_test)
  elif model_name == 'vgg16':
    x_tr = vgg16.preprocess_input(x_train)
    x_te = vgg16.preprocess_input(x_test)
  else:
    raise ValueError('unknown model_name (typo?):', model_name)

  boundry = int(len(x_tr)*0.9)
  x_val = x_tr[boundry:]
  x_tr = x_tr[:boundry]
  y_val = y_train[boundry:]
  y_tr = y_train[:boundry]

  # create model
  model, base_model, new_layers = create_model(model_name, nlayers, units, dropout)

  # Freezing the model
  base_model.trainable = False
  new_layers.trainable = True

  # train
  es = EarlyStopping(patience=patience,  restore_best_weights=True, monitor="val_accuracy", baseline=0.5)
  model.compile(optimizer=Adam(learning_rate=lr, amsgrad=True), loss=loss(), metrics=['accuracy'])
  history = model.fit(x_tr, y_tr, batch_size=batch_size, epochs=epochs1, validation_data=(x_val, y_val), callbacks=[es])
  t_end1 = perf_counter()

  # save & evaluate
  if eval:
    model.save(f'{model_name}_transformed.h5')
    print(f"\nResults for the training of top layer of {model_name}:\n")
    print(f'time: {t_end1 - t_start:.4f}secs')
    model.summary()
    print_report(model, x_te, y_test, label_names)
    print_learning_curves(history)

  # Freeze all the layers before the `fine_tune_at` layer
  new_layers.trainable = True
  base_model.trainable = True
  fine_tune_at = len(base_model.layers) // 2
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  # train
  model.compile(optimizer=Adam(learning_rate=lr/10, amsgrad=True), loss=loss(), metrics=['accuracy'])
  history = model.fit(x_tr, y_tr, batch_size=batch_size, epochs=epochs2, validation_data=(x_val, y_val), callbacks=[es])
  t_end2 = perf_counter()

  # save & evaluate
  if eval:
    model.save(f'{model_name}_tuned.h5')
    print(f"\nResults for the fine tunig of top layer of {model_name}:\n")
    print(f'time: {t_end2 - t_start:.4f}secs')
    print_report(model, x_te, y_test, label_names)
    print_learning_curves(history)


  # result
  acc = max(history.history['val_accuracy'])
  print(f'Final accuracy for model: {acc}\n')
    
  return -1 * acc # score for minimization

In [ ]:
def optimize_params(model_name, nlayers, reuse_params=None, opti_iter=10):

  space = [
    Integer(16, 512, prior='log-uniform', base=2,  name='units'),
    Real(0.0, 0.6,   prior='uniform',     base=10, name='dropout_rate'),
    Real(1e-5, 1e-3, prior='log-uniform', base=10, name='learning_rate'),  # reuse
    Integer(4, 6,    prior='uniform',     base=10, name='batch_size'),     # reuse
    Categorical([CategoricalCrossentropy, KLDivergence], name='loss')      # reuse
  ]

  if reuse_params:  # remove lr, bs, loss from space
    space = space[:2]

  fct = lambda params: train_model(params, model_name, nlayers, reuse_params)
  cp_fname = f"{gdrive_dir}bo_checkpoint_{model_name}_layers{nlayers}.pkl"

  # file already exists - optimization has been interrupted
  if os.path.exists(cp_fname):
    res = skopt.load(cp_fname)
    x0 = res.x_iters
    y0 = res.func_vals
    init_points=0
    print("imported parameters", x0)
    print("imported values", y0)
    opti_iter = opti_iter - min(opti_iter,len(y0))
  else:
    x0=None
    y0=None
    init_points=4

  res = gp_minimize(fct,                          # the function to minimize
                    space,                        # the bounds on each dimension of x
                    n_initial_points=init_points, # initial random points
                    x0=x0,                        # Initial input points
                    y0=y0,                        # evaluation of x0
                    n_calls=opti_iter,            # the number of evaluations of f
                    callback=[CheckpointSaver(cp_fname, compress=9)],
                    random_state=420)             # the random seed

  # evaluate
  print(f"\nResults for the Baysian Optimization for {model_name}:")
  print('best parameters: ', res.x)
  print('best accuracy: ', -res.fun)
  plot_convergence(res)
  plt.plot()

  return res

In [ ]:
# # optimize layers=1
# resNet50_res = optimize_params("resNet50", nlayers=1, opti_iter=20)
# units_resNet50 = resNet50_res.x[0]
# acc = -resNet50_res.fun

# # optimize next layers
# for layer in range(2, 5):
#   print('\n_________________________\nNumber of layers:', layer)
#   new_resNet50_res = optimize_params("resNet50", nlayers=layer, units=units_resNet50)
#   new_acc = -new_resNet50_res.fun

#   if new_acc < acc: break
#   acc = new_acc
#   resNet50_res = new_resNet50_res

# final_nlayers = layer-1
# print('\nFinal number of layers:',final_nlayers)

# train_model(resNet50_res.x[:2], "resNet50", nlayers=final_nlayers, units=units_resNet50, eval=True)
# final_resNet50 = load_model(f'resNet50_tuned.h5')
# final_resNet50.save(f'{gdrive_dir}final_resNet50.h5')

In [ ]:
# optimize layers=1
inceptionv3_res = optimize_params("inceptionv3", nlayers=1, opti_iter=35)
lr_v3, bs_v3, loss_v3 = inceptionv3_res.x[2:]  # reuse
acc = -inceptionv3_res.fun

# optimize next layers
for layer in range(2, 5):
  print('\n_________________________\nNumber of layers:', layer)
  new_inceptionv3_res = optimize_params("inceptionv3", nlayers=layer, 
                                     reuse_params=[lr_v3, bs_v3, loss_v3], 
                                     opti_iter=15)
  new_acc = -inceptionv3_res.fun
  print(new_acc, acc)
  if new_acc < acc: break
  acc = new_acc
  inceptionv3_res = new_inceptionv3_res

final_nlayers = layer-1
print('\nFinal number of layers:',final_nlayers)

# evaluate & save in drive
train_model(inceptionv3_res.x, "inceptionv3", nlayers=final_nlayers, reuse_params=[lr_v3, bs_v3, loss_v3], eval=True)
final_inceptionv3 = load_model(f'inceptionv3_tuned.h5')
final_inceptionv3.save(f'{gdrive_dir}final_inceptionv3.h5')

In [ ]:
# # optimize layers=1
# vgg16_res = optimize_params("vgg16", nlayers=1, opti_iter=20)
# units_vgg16 = vgg16_res.x[0]
# acc = -vgg16_res.fun

# # optimize next layers
# for layer in range(2, 5):
#   print('\n_________________________\nNumber of layers:', layer)
#   new_vgg16_res = optimize_params("vgg16", nlayers=layer, units=units_vgg16)
#   new_acc = -vgg16_res.fun

#   if new_acc < acc: break
#   acc = new_acc
#   vgg16_res = new_vgg16_res

# final_nlayers = layer-1
# print('\nFinal number of layers:',final_nlayers)

# #save in g-drive
# train_model(vgg16_res.x, "vgg16", nlayers=final_nlayers, units=units_vgg16, eval=True)
# final_vgg16 = load_model(f'vgg16_tuned.h5')
# final_vgg16.save(f'{gdrive_dir}final_vgg16.h5')

In [ ]:
# # optimize with one layers
# resNet50_res = optimize_params("resNet50", nlayers=1, opti_iter=35)
# lr_resNet50, bs_resNet50, loss_resNet50 = resNet50_res.x[2:]  # reuse
# acc = -resNet50_res.fun

# # optimize next layers
# for layer in range(2, 5):
#   print('\n_________________________\nNumber of layers:', layer)
#   new_resNet50_res = optimize_params("resNet50", 
#                                      nlayers=layer, 
#                                      reuse_params=[lr_resNet50, bs_resNet50, loss_resNet50], 
#                                      opti_iter=15)
#   new_acc = -new_resNet50_res.fun

#   if new_acc < acc: break
#   acc = new_acc
#   resNet50_res = new_resNet50_res

# final_nlayers = layer-1
# print('\nFinal number of layers:', final_nlayers)

# # evaluate
# # train_model(params, model_name, nlayers, reuse_params=None, eval=False):
# train_model(resNet50_res.x[:2], 
#             "resNet50", 
#             nlayers=final_nlayers, 
#             reuse_params=[lr_resNet50, bs_resNet50, loss_resNet50], 
#             eval=True)
# final_resNet50 = load_model(f'resNet50_tuned.h5')
# final_resNet50.save(f'{gdrive_dir}final_resNet50.h5')